# **MODEL 1**

In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bokeh')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
df = pd.read_csv('/dataset.csv')
df

In [ ]:
df.isna().sum()

In [ ]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

In [ ]:
BASE_PRICE = 10.0
α = 0.5
df["price"] = BASE_PRICE + α * df["OccupancyRate"].cumsum()
df["price"] = df["price"].clip(lower=0.5*BASE_PRICE, upper=2*BASE_PRICE)

In [ ]:
df

In [ ]:
df[["SystemCodeNumber","Timestamp","Occupancy","Capacity","OccupancyRate","price"]].to_csv(
    "parking_stream.csv", index=False
)

In [ ]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    OccupancyRate: float

In [ ]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d"),
    OccupancyRate = data.Occupancy / data.Capacity,
    day_id = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d") + "_" + data.SystemCodeNumber,
)

In [ ]:
id_lookup = (
    data_with_time
    .select(pw.this.day_id, pw.this.SystemCodeNumber)
    .groupby(pw.this.day_id)
    .reduce(
        day_id = pw.reducers.min(pw.this.day_id),
        SystemCodeNumber = pw.reducers.min(pw.this.SystemCodeNumber)
    )
)

In [ ]:
import datetime
reduced = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day_id,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.OccupancyRate),
        occ_min = pw.reducers.min(pw.this.OccupancyRate),
        cap = pw.reducers.max(pw.this.Capacity),
        day_id = pw.reducers.min(pw.this.day_id),
        lot = pw.reducers.min(pw.this.SystemCodeNumber)
    )
)
delta_window = reduced.with_columns(
    price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
)

In [ ]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="navy")

    fig.circle("t", "price", source=source, size=6, color="red")

    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")

pn.Column(viz).servable()
viz.servable()

In [ ]:
pw.io.jsonlines.write(
    delta_window.select(
        pw.this.t,
        pw.this.lot,
        pw.this.price
    ),
    "pricing_output.jsonl"
)

In [ ]:
pw.run()

In [ ]:
df_price = pd.read_json("pricing_output.jsonl", lines=True)
df_price["t"] = pd.to_datetime(df_price["t"])

In [ ]:
from bokeh.plotting import figure, show, output_notebook , save , output_file
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10

output_notebook()

plots = []
lots = sorted(df_price["lot"].unique())
palette = Category10[10] + Category10[4]

for i, lot in enumerate(lots):
    df_lot = df_price[df_price["lot"] == lot]
    source = ColumnDataSource(df_lot)

    fig = figure(
        height=300,
        width=800,
        title=f"Price Trend - Lot {lot}",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, color=palette[i % len(palette)], line_width=2)
    fig.scatter("t", "price", source=source, size=5, marker="circle", color="black")
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"

    plots.append(fig)

print(f"Generated {len(plots)} plots for lots: {lots}")
show(column(*plots))

In [ ]:
df = pd.read_json("pricing_output.jsonl", lines=True)
df["t"] = pd.to_datetime(df["t"])

plots = []
lots = sorted(df["lot"].unique())
palette = Category10[10] + Category10[4]

for i, lot in enumerate(lots):
    df_lot = df[df["lot"] == lot]
    source = ColumnDataSource(df_lot)

    fig = figure(
        height=300,
        width=800,
        title=f"Price Trend - Lot {lot}",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, color=palette[i % len(palette)], line_width=2)
    fig.scatter("t", "price", source=source, color="black", size=5, marker="circle")
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"

    plots.append(fig)

output_file("all_lot_prices.html")
save(column(*plots))

In [ ]:
from bokeh.models import Legend
df = pd.read_json("pricing_output.jsonl", lines=True)

df["t"] = pd.to_datetime(df["t"])
df["lot"] = df["lot"].astype(str)

fig = figure(
    height=500,
    width=900,
    title="Daily Pricing Across Parking Lots",
    x_axis_type="datetime"
)

lots = sorted(df["lot"].unique())
for i, lot in enumerate(lots):
    df_lot = df[df["lot"] == lot]
    source = ColumnDataSource(df_lot)

    fig.line("t", "price", source=source, legend_label=f"Lot {lot}",
             color=palette[i % len(palette)], line_width=2)

fig.xaxis.axis_label = "Time"
fig.yaxis.axis_label = "Price ($)"
fig.add_layout(fig.legend[0], 'right')
fig.legend.label_text_font_size = "8pt"

output_file("lot_price_plot_from_json.html")
save(fig)